In [1]:
from haystack.file_converter.pdf import PDFToTextConverter
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])

In [2]:
book = converter.convert(file_path="/home/elena/Downloads/9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf", meta=None)

In [4]:
type(book), len(book), book.keys(), len(book["text"]), type(book["meta"])

(dict, 2, dict_keys(['text', 'meta']), 579763, NoneType)

In [7]:
book["text"][1500:5000]

'Buckingham,\nMegan Carlisle, Mahesh Dhyani, Manasa Kumar, Alex Mazonowicz, Bridget Neale,\nDominic Pereira, Shiny Poojary, Abhishek Rane, Brendan Rodrigues, Mugdha Sawarkar,\nErol Staveley, Ankita Thakur, Nitesh Thakur, and Jonathan Wray\nFirst published: April 2019\nSecond edition: February 2020\nProduction reference: 1270220\nPublished by Packt Publishing Ltd.\nLivery Place, 35 Livery Street\n\x0cTable of Contents\nPreface\u202f\t\u202fi\nChapter 1: Introduction to Machine Learning with Keras\u202f\t\u202f1\nIntroduction ..................................................................................................... 2\nData Representation ...................................................................................... 4\nTables of Data ........................................................................................................ 4\nLoading Data ......................................................................................................... 5\nExercise 1

In [3]:
from haystack.preprocessor.preprocessor import PreProcessor

# we can use PreProcessor to split by passage, sentence and word
# passage wil split data, i.e. book in to dicts; one with meta _split_id 0 and the other _split_id 1 
# sentence with split length 260 will split book in 14 dicts; key meta numerating those dicts (from 0 to 13)
# word with split length 260 will split book into 329 dicts

In [4]:
from haystack.preprocessor.preprocessor import PreProcessor

processor_passage = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="passage",
                         split_respect_sentence_boundary=False)

[nltk_data] Downloading package punkt to /home/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
passage = processor_passage.process(book)

In [12]:
type(passage), len(passage), len(passage[0]), len(passage[1]), type(passage[0]), passage[0].keys(), passage[1].keys()

(list, 2, 2, 2, dict, dict_keys(['text', 'meta']), dict_keys(['text', 'meta']))

In [16]:
len(passage[0]["text"]), len(passage[1]["text"]), len(passage[0]["text"])+len(passage[1]["text"]), passage[0]["meta"], passage[1]["meta"]

(369553, 206463, 576016, {'_split_id': 0}, {'_split_id': 1})

In [6]:
processor_sentence = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="sentence",
                         split_length = 260,
                         split_respect_sentence_boundary=True)
                    

[nltk_data] Downloading package punkt to /home/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
sent = processor_sentence.process(book)

In [19]:
type(sent), len(sent), type(sent[0]), sent[0].keys(), len(sent[0]["text"]), len(sent[9]["text"])

(list, 14, dict, dict_keys(['text', 'meta']), 53981, 30844)

In [8]:
processor_word = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="word",
                         split_length = 260,
                         split_respect_sentence_boundary=True)

[nltk_data] Downloading package punkt to /home/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
word=processor_word.process(book)

A sentence found with word count higher than the split length.


In [22]:
type(word), len(word), word[0].keys(), word[0]["meta"]

(list, 329, dict_keys(['text', 'meta']), {'_split_id': 0})

In [21]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()
document_store.delete_all_documents()

In [22]:
document_store.write_documents(sent)

In [23]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=False,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
    

12/02/2020 09:27:43 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/02/2020 09:27:51 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [24]:
document_store.update_embeddings(retriever)

ls | 403\n8. Check the shape of the image. It should be (1,1000):\ny_pred.shape\n\nThe preceding code provides the following output:\n\n9. Select the top five probabilities of what our image is using the decode_\npredictions function and by passing the predictor variable, y_pred, as the\nargument and the top number of predictions and corresponding labels:\nfrom keras.applications.resnet50 import decode_predictions\n\nThe preceding code provides the following output:\n\nThe first column of the array is an internal code number. The second is the label,\nwhile the third is the probability of the image matching the label. 10. Put the predictions in a human-readable format. Print the most probable label\nfrom the output from the result of the decode_predictions function:\nlabel = decode_predictions(y_pred)\n# Most likely result is retrieved, for example, the highest\nprobability\ndecoded_label = label[0][0]\n# The classification is printed\n\nThe preceding code produces the following output

TypeError: 'NoneType' object is not iterable

In [25]:
from haystack.reader.farm import FARMReader
farm_reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

12/02/2020 09:28:01 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/02/2020 09:28:01 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/02/2020 09:28:04 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/02/2020 09:28:13 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/02/2020 09:28:13 - INFO - farm.inf

In [11]:
from haystack import Finder

In [26]:
finder = Finder(farm_reader, retriever)

In [27]:
prediction = finder.get_answers(question="layer", top_k_retriever=1, top_k_reader=5)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.86 Batches/s]
12/02/2020 09:28:20 - INFO - haystack.finder -   Got 0 candidates from retriever
12/02/2020 09:28:20 - INFO - haystack.finder -   Retriever did not return any documents. Skipping reader ...


In [28]:
from haystack.utils import print_answers

In [29]:
print_answers(prediction, details="medium")

[]


In [31]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

12/02/2020 09:28:50 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.042s]
12/02/2020 09:28:50 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.003s]
12/02/2020 09:28:50 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.030s]
12/02/2020 09:28:50 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.004s]


In [37]:
document_store.index

'document'

In [38]:
document_store.delete_all_documents(index="docuement")

12/02/2020 09:33:11 - INFO - elasticsearch -   POST http://localhost:9200/docuement/_delete_by_query [status:404 request:0.036s]


In [39]:
document_store.write_documents(sent)

12/02/2020 09:33:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.595s]


In [40]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [41]:
finder = Finder(farm_reader, retriever)

In [42]:
prediction = finder.get_answers(question="What is RNN?", top_k_retriever=10, top_k_reader=5)

12/02/2020 09:33:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.035s]
12/02/2020 09:33:20 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 09:33:20 - INFO - haystack.finder -   Reader is looking for detailed answer in 143935 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.73s/ Batches]


In [43]:
print_answers(prediction, details="medium")

[   {   'answer': 'Recurrent Neural Networks',
        'context': ' chapter, you will\n'
                   'be able to build sequential models, explain Recurrent '
                   'Neural Networks (RNNs),\n'
                   'describe the vanishing gradient problem, and implemen',
        'score': 16.782752990722656},
    {   'answer': 'Recurrent Neural Networks',
        'context': ' chapter, you will\n'
                   'be able to build sequential models, explain Recurrent '
                   'Neural Networks (RNNs),\n'
                   'describe the vanishing gradient problem, and implemen',
        'score': 16.782752990722656},
    {   'answer': 'Recurrent Neural Networks',
        'context': 'rthermore, we will learn\n'
                   'how sequential modeling is related to Recurrent Neural '
                   'Networks (RNN).We will\n'
                   'learn about the vanishing gradient problem in d',
        'score': 16.642990112304688},
    {   'answer': 'Rec

In [44]:
print_answers(finder.get_answers(question="What is a layer?", top_k_retriever=10, top_k_reader=5), details="medium")

12/02/2020 09:36:05 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.021s]
12/02/2020 09:36:05 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 09:36:05 - INFO - haystack.finder -   Reader is looking for detailed answer in 152889 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:27<00:00, 13.95s/ Batches][   {   'answer': 'user-defined layer',
        'context': 'tially, what we will do is\n'
                   'replace the last layer of VGG16 with a user-defined '
                   'layer.Before we begin, ensure you have downloaded the '
                   'image datasets fr',
        'score': 10.711793899536133},
    {   'answer': 'user-defined layer',
        'context': 'tially, what we will do is\n'
                   'replace the last layer of VGG16 with a user-defined layer. '
                   'Before we begin, ensure you have downloaded the image '
                   'datasets f',
        'score': 7.761

In [45]:
document_store.delete_all_documents(index="docuement")
document_store.write_documents(word)
retriever = ElasticsearchRetriever(document_store=document_store)
finder = Finder(farm_reader, retriever)

12/02/2020 09:38:10 - INFO - elasticsearch -   POST http://localhost:9200/docuement/_delete_by_query [status:404 request:0.006s]
12/02/2020 09:38:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.966s]


In [46]:
prediction = finder.get_answers(question="What is RNN?", top_k_retriever=10, top_k_reader=5)

12/02/2020 09:38:14 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.022s]
12/02/2020 09:38:14 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 09:38:14 - INFO - haystack.finder -   Reader is looking for detailed answer in 83918 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:27<00:00, 13.82s/ Batches]


In [47]:
print_answers(prediction, details="medium")

[   {   'answer': 'Recurrent Neural Networks',
        'context': 'rthermore, we will learn\n'
                   'how sequential modeling is related to Recurrent Neural '
                   'Networks (RNN).We will\n'
                   'learn about the vanishing gradient problem in d',
        'score': 16.642990112304688},
    {   'answer': 'Recurrent Neural Networks',
        'context': 'rthermore, we will learn\n'
                   'how sequential modeling is related to Recurrent Neural '
                   'Networks (RNN).We will\n'
                   'learn about the vanishing gradient problem in d',
        'score': 16.642990112304688},
    {   'answer': 'Recurrent Neural Networks',
        'context': ' chapter, you will\n'
                   'be able to build sequential models, explain Recurrent '
                   'Neural Networks (RNNs),\n'
                   'describe the vanishing gradient problem, and implemen',
        'score': 15.48896312713623},
    {   'answer': 'Recu